Création et enregistrement des dictionnaires d'occurences par classe

In [ ]:
import csv
import re
from collections import defaultdict, Counter

types = defaultdict(Counter)
belonging = defaultdict(lambda: defaultdict(int))

def sentences_to_list_of_words(text):
    return re.findall(r'\b\w+\b', text.lower())

with open('company-document-text.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    type_nb = {
        "invoice" : 0,
        "purchase Order": 0,
        "report":0,
        "ShippingOrder":0
    }
    for idx, line in enumerate(reader):
        label = line["label"]
        type_nb[label] += 1
        words = sentences_to_list_of_words(line["text"])
        unique_words = set(words)  # Pour éviter de compter 2 fois le même mot dans une ligne

        types[label].update(words)  # Occurrence totale

        for word in unique_words:
            belonging[word][label] += 1  # Appartenance (nb de lignes)

# Écriture triée
for label in types:
    filename = f"./occurences/{label}_output.csv"
    word_data = []

    for word, freq in types[label].items():
        b = belonging[word][label]
        score = freq * b
        word_data.append((word, freq, b, score))

    # Tri rapide avec Python natif
    word_data.sort(key=lambda x: (-x[2], -x[1]))  # Tri par appartenance décroissante puis fréquence

    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Word', 'Frequency', 'Belonging', 'Score'])
        writer.writerows(word_data)

print(type_nb)


Filtre à + ou - 20%

In [ ]:
import csv

file = "./occurences"
percent_1 = 0.20

total_invoice = type_nb['invoice']
total_po = type_nb['purchase Order']
total_report = type_nb['report']
total_shipping = type_nb['ShippingOrder']

def load_word_counts(filename):
    counts = {}
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            word = row['Word']
            count = int(row['Belonging'])
            counts[word] = count
    return counts

def is_within_margin(x, y):
    margin = x * percent_1
    return (x - margin) <= y <= (x + margin)

# Chargement des dictionnaires de chaque type
type1 = load_word_counts(f'{file}/invoice_output.csv')
type2 = load_word_counts(f'{file}/purchase Order_output.csv')
type3 = load_word_counts(f'{file}/report_output.csv')
type4 = load_word_counts(f'{file}/ShippingOrder_output.csv')

# Résultat : mots à supprimer
mots_trop_communs = []
mots_trop_rares = set()

percent_2 = 0.10

# Détection des mots trop rares (moins de 10% du nombre de fichiers par catégorie)
for word, count in type1.items():
    if count < total_invoice * percent_2:
        mots_trop_rares.add(word)

for word, count in type2.items():
    if count < total_po * percent_2:
        mots_trop_rares.add(word)

for word, count in type3.items():
    if count < total_report * percent_2:
        mots_trop_rares.add(word)

for word, count in type4.items():
    if count < total_shipping * percent_2:
        mots_trop_rares.add(word)

# Supprimer les mots (de la liste des mots à supprimer) qui apparaissent dans plus de 10% des fichiers d'une catégorie
for word in list(mots_trop_rares):
    if (word in type1 and type1[word] > total_invoice * percent_2) or \
       (word in type2 and type2[word] > total_po * percent_2) or \
       (word in type3 and type3[word] > total_report * percent_2) or \
       (word in type4 and type4[word] > total_shipping * percent_2):
        mots_trop_rares.remove(word)

# Résultat : mots trop communs
for word in type1:
    if word in type2 and word in type3 and word in type4:
        c1 = type1[word]
        c2 = type2[word]
        c3 = type3[word]
        c4 = type4[word]

        if all(is_within_margin(c1, c) for c in [c2, c3, c4]):
            mots_trop_communs.append(word)

mots_trop_rares = list(mots_trop_rares)

print(f"{len(mots_trop_communs)} mots trop communs trouvés.")
print(f"{len(mots_trop_rares)} mots trop rares trouvés.")


nettoyage des fichiers csv on enlève les mots trop commun

In [ ]:
import csv
import os

file2 = "./occurences/nettoyer"

mots_a_supprimer = mots_trop_rares + mots_trop_communs

def nettoyer_fichier(input_filename, output_filename, mots_a_supprimer):
    with open(input_filename, newline='', encoding='utf-8') as infile, \
         open(output_filename, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            if row['Word'] not in mots_a_supprimer:
                writer.writerow(row)

# Fichiers à nettoyer
fichiers = [
    f'{file}/invoice_output.csv',
    f'{file}/purchase Order_output.csv',
    f'{file}/report_output.csv',
    f'{file}/ShippingOrder_output.csv'
]

for f in fichiers:
    nom_fichier = os.path.basename(f).replace('.csv', '_nettoye.csv')
    chemin_sortie = os.path.join(file2, nom_fichier)
    nettoyer_fichier(f, chemin_sortie, mots_a_supprimer)

print("Nettoyage terminé ! Fichiers sauvegardés dans le dossier 'nettoyer'.")


Création du Dataset d'entrainement

In [ ]:
import csv

file = "./occurences"
file2 = "./occurences/nettoyer"
fichier_principal = "company-document-text.csv"
fichier_mots = [
    f'{file2}/invoice_output_nettoye.csv',
    f'{file2}/purchase Order_output_nettoye.csv',
    f'{file2}/report_output_nettoye.csv',
    f'{file2}/ShippingOrder_output_nettoye.csv'
]
fichier_sortie = "training_data_set.csv"
colonne_texte = "Word"
pourcentage = 0.8
max_words = 800

def load_words_from_file(file_path):
    words = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            word = row['Word']
            words.append(word)
            count += 1
            if count >= max_words:
                break
    return words

# Charger tous les mots des fichiers spécifiés
mots = []
for fichier in fichier_mots:
    mots += load_words_from_file(fichier)

# Créer la liste des features, y compris les mots extraits
features = list(set(mots)) + ['word_count', 'invoice', 'purchase Order', 'report', 'ShippingOrder']

print(len(features))

# Variables pour accumuler les sommes des colonnes
somme_colonnes = {feature: 0 for feature in features[1:-1]}  # Exclure 'text' et 'word_count'

# Lire et modifier le fichier CSV principal
with open(fichier_principal, mode='r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    
    # Créer un fichier de sortie avec les modifications
    with open(fichier_sortie, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=features)
        writer.writeheader()
        
        for row in reader:
            texte = row[reader.fieldnames[0]]  # Récupérer le texte dans la première colonne
            output_row = {feature: 0 for feature in features}

            # Extraire les mots avec re
            mots_dans_texte = re.findall(r'\b\w+\b', texte.lower())

            # Compter les mots pertinents
            for mot in mots_dans_texte:
                if mot in mots:
                    output_row[mot] += 1

            # Ajouter le label
            if row['label'] in output_row:
                output_row[row['label']] = 1

            # Word count (à convertir si nécessaire)
            output_row['word_count'] = int(row['word_count'])

            # Mettre à jour les sommes
            for feature in somme_colonnes:
                somme_colonnes[feature] += int(output_row[feature])

            # Écrire la ligne modifiée
            writer.writerow(output_row)

# Afficher la somme de chaque colonne
print("Somme des colonnes :")
for feature, total in somme_colonnes.items():
    print(f"{feature}: {total}")

print(f"Le fichier {fichier_sortie} a été créé avec succès.")


Création du modèle XGBOOST

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Charger ton jeu de données
data = pd.read_csv("training_data_set.csv")

# Supposons que chaque étiquette (label) est sous forme de quadruplet (1 ou 0 pour chaque type)
# Exemple : 'invoice', 'purchase Order', 'report', 'ShippingOrder'
X = data.drop(columns=['invoice', 'purchase Order', 'report', 'ShippingOrder'])  # Les caractéristiques
y = data[['invoice', 'purchase Order', 'report', 'ShippingOrder']]  # Les 4 colonnes de labels

# Diviser en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser le modèle XGBoost et l'encoder dans un classificateur multi-output
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# MultiOutputClassifier permet de traiter plusieurs sorties simultanément
model = MultiOutputClassifier(xgb_model, n_jobs=-1)

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions sur le jeu de test
y_pred = model.predict(X_test)

# Évaluer les performances du modèle
print("Précision globale : ", accuracy_score(y_test, y_pred))
print("Rapport de classification :\n", classification_report(y_test, y_pred))

# Enregistre ton modèle entraîné dans un fichier
joblib.dump(model, "xgboost_modele.pkl")


Réutilisation du modèle pour un fichier donnée

In [ ]:
import joblib

# Charger le modèle sauvegardé
model = joblib.load("xgboost_modele.pkl")

vocab = list(set(mots))

def vectorize(doc, vocab):
    word_total = 0
    vec = [0] * len(vocab)
    for word in doc:
        word_total += 1
        if word in vocab:
            vec[vocab.index(word)] += 1
    vec.append(word_total)
    return vec

# Exemple de document brut
texte = 'order id  10326 shipping details  ship name  bólido comidas preparadas ship address  c  araquil, 67 ship city  madrid ship region  southern europe ship postal code  28023 ship country  spain customer details  customer id  bolid customer name  bólido comidas preparadas employee details  employee name  margaret peacock shipper details  shipper id  2 shipper name  united package order details  order date  2016-10-10 shipped date  2016-10-14 products  -------------------------------------------------------------------------------------------------- product  chef anton s cajun seasoning quantity  24 unit price  17 6 total  422 40000000000003 -------------------------------------------------------------------------------------------------- product  ravioli angelo quantity  16 unit price  15 6 total  249 6 -------------------------------------------------------------------------------------------------- product  rhönbräu klosterbier quantity  50 unit price  6 2 total  310 0 total price  total price  982 0'
mots_dans_texte = re.findall(r'\b\w+\b', texte.lower())

# Vectorisation
x = vectorize(mots_dans_texte, vocab)
print(x)
# === 4. Prédiction ===

y_pred = model.predict([x])[0]  # [x] car il attend une liste de vecteurs

# === 5. Interprétation du résultat ===

labels = ["invoice", "purchase Order", "report", "ShippingOrder"]
predicted_labels = [labels[i] for i, v in enumerate(y_pred) if v == 1]

# === 6. Affichage du résultat ===

if predicted_labels:
    print("✅ Type de document détecté :", predicted_labels[0])
else:
    print("❌ Aucun type détecté")

